# 📓 The GenAI Revolution Cookbook

**Title:** How to Build an LLM Agent from Scratch with GPT-4 ReAct

**Description:** Build a fully functional LLM agent in Python using ReAct, tool actions, regex parsing, and GPT-4, automated control loop included.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Here’s an improved version of your draft content. I’ve added an introduction to situate this as a fundamental demonstration, linked it explicitly to the *ReAct* paper (arXiv:2210\.03629\) with key insights, and preserved your original technical structure. I also applied humanization rules for clarity and flow.

---

## Demonstrating What AI Agents Really Are

You probably use agent frameworks like LangChain or BabyAGI without really seeing how they work under the hood. This tutorial is a chance to peel back the layers. You’re going to build a ReAct\-style agent from scratch. This will help you understand what AI agents fundamentally are, how they reason, act, and loop with tools—all without any third\-party framework. When you see how each part works, you’ll understand more clearly what those frameworks are doing for you.

The core idea comes from the paper *“ReAct: Synergizing Reasoning and Acting in Language Models”* (arXiv:2210\.03629\) by Yao et al. ([arxiv.org](https://arxiv.org/abs/2210.03629?utm_source=openai)) ReAct shows that you can get better performance and interpretability when a model both reasons (writes *Thoughts*) and acts (calls tools), weaving those steps together with observations. ([arxiv.org](https://arxiv.org/abs/2210.03629?utm_source=openai))

---

## Introduction

Building an agent that can reason, call tools, and iterate without supervision is now straightforward—if you control the format, validate the inputs, and guard against runaway loops. This tutorial shows you how to build a ReAct agent from scratch using the OpenAI Python SDK, three simple tools (distance lookup, travel time calculation, sum), a regex\-based action parser, and a single\-action\-per\-turn control loop with a max\-turn guard. You’ll end with a runnable, testable agent that can answer multi\-step questions like “How long to drive from Montreal to Boston at 60 mph?”

**Prerequisites:** Python 3\.10\+, an OpenAI API key, and basic familiarity with LLMs. Token usage is minimal. Using `gpt-4o-mini` for tool lookups keeps costs low. This tutorial is Colab\-ready and starts with a `!pip install` cell.

---

## Why This Approach Works

Frameworks add convenience—but they hide behavior. If you build things from scratch, you gain complete control. You can debug faster. You’ll know exactly when and why the model does something.

**Why GPT\-4o and GPT\-4o\-mini?**
GPT\-4o provides strong reasoning for the main agent loop. GPT\-4o\-mini handles simple tool lookups cheaply and quickly. The combination keeps cost and latency balanced without losing reliability.

**Why ReAct?**
The ReAct pattern (Reason \+ Act) forces the model to articulate its reasoning before taking action. That makes behavior interpretable and easier to debug. A strict format—Thought → Action → PAUSE → Observation—lets you parse and validate every step programmatically.

**Why regex parsing?**
Regex is deterministic, fast, and transparent. You know exactly what matches and what does not. For production you could swap in JSON\-based arguments or the OpenAI function\-call API. Regex is a simple way to start.

---

## How It Works (High\-Level Overview)

1. **Agent receives a question**, then generates a Thought and an Action. Example: `lookup_distance[Montreal, Boston]`.
2. **Control loop parses the Action** using regex, validates it, and calls its Python function.
3. **Tool returns a result** (e.g. `308 miles`), which becomes the Observation.
4. **Agent updates reasoning** with the new info. It either calls another tool or gives a final Answer.
5. **Loop stops** once the agent outputs `Answer:` or hits the max turn limit.

---

## Setup \& Installation

Run this cell in Colab or your local environment to install dependencies:

In [ ]:
!pip install --upgrade openai python-dotenv

Set your OpenAI API key. In Colab:

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key

Or create a `.env` file locally:

In [ ]:
OPENAI_API_KEY=sk-...

Verify the key is set before proceeding:

In [ ]:
import os
if not os.getenv("OPENAI_API_KEY"):
    raise EnvironmentError("OPENAI_API_KEY not set. Please set it before running.")

---

## Step\-by\-Step Implementation

### Define the System Prompt

This is the contract between you and the model. It enforces the ReAct format and lists available actions with exact signatures.

In [ ]:
REACT_SYSTEM_PROMPT = """You are a ReAct-style agent. Follow this exact format:

Thought: describe your reasoning briefly.
Action: function_name[param1, param2, ...]
PAUSE

After you receive an Observation, continue:

Thought: update your reasoning briefly.
(Optional another) Action: function_name[…]
PAUSE

When done, provide:
Answer: <final answer, concise and complete>

Rules:
- At most ONE Action per turn.
- If no action is needed, go directly to Answer.
- Use only these actions with exact signatures:
  - lookup_distance[location1, location2]
  - calculate_travel_time[distance, speed]
  - calculate_sum[value1, value2]
- Use miles for distance and mph for speed unless specified.
- Wait for Observation after PAUSE; do not fabricate results.
"""

---

### Build the Agent Class

The `Agent` class manages conversation history. It makes calls to the OpenAI API. It keeps track of messages so the model has full context each turn.

In [ ]:
import os
import logging
from dataclasses import dataclass, field
from typing import List, Dict
from openai import OpenAI

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

@dataclass
class Agent:
    system_prompt: str
    model: str = "gpt-4o"
    temperature: float = 0.0
    messages: List[Dict[str, str]] = field(default_factory=list)

    def __post_init__(self):
        self.messages = [{"role": "system", "content": self.system_prompt}]

    def __call__(self, user_content: str) -> str:
        self.messages.append({"role": "user", "content": user_content})
        return self.execute()

    def execute(self) -> str:
        resp = client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=self.messages,
        )
        content = resp.choices[0].message.content
        self.messages.append({"role": "assistant", "content": content})
        logger.debug(f"Assistant response: {content}")
        return content

---

### Implement the Tools

Each tool is a simple Python function. Distance lookup uses an LLM call for realism. You can swap in deterministic maps or APIs later.

In [ ]:
import re

def generate_response(prompt: str, model: str = "gpt-4o-mini") -> str:
    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise assistant. Reply with the answer only."},
            {"role": "user", "content": prompt},
        ],
    )
    return resp.choices[0].message.content.strip()

def lookup_distance(location1: str, location2: str) -> str:
    prompt = (
        f"What is the typical driving distance in miles between {location1} and {location2}? "
        "Return a single number followed by ' miles'. No extra text."
    )
    ans = generate_response(prompt)
    m = re.search(r"([0-9]+(?:.[0-9]+)?)\\s*miles", ans, re.IGNORECASE)
    if not m:
        ans = generate_response(
            f"Return only the distance number in miles for driving between {location1} and {location2} in the format '<number> miles'."
        )
        m = re.search(r"([0-9]+(?:.[0-9]+)?)\\s*miles", ans, re.IGNORECASE)
    return f"{m.group(1)} miles" if m else "unknown miles"

def _extract_number(s: str) -> float:
    m = re.search(r"(-?\\d+(?:\.\\d+)?)", s)
    if not m:
        raise ValueError(f"Cannot parse number from: {s}")
    return float(m.group(1))

def calculate_travel_time(distance: str, speed: str) -> str:
    d = _extract_number(distance)
    v = _extract_number(speed)
    if v == 0:
        return "infinite hours"
    hours = d / v
    return f"{round(hours, 2)} hours"

def _extract_number_and_unit(s: str) -> tuple:
    m = re.search(r"(-?\\d+(?:\.\\d+)?)\\s*([a-zA-Z/%]+)?", s.strip())
    if not m:
        raise ValueError(f"Cannot parse: {s}")
    value = float(m.group(1))
    unit = m.group(2) or ""
    return value, unit

def calculate_sum(value1: str, value2: str) -> str:
    v1, u1 = _extract_number_and_unit(value1)
    v2, u2 = _extract_number_and_unit(value2)
    unit = u1 if u1 == u2 else ""
    total = v1 + v2
    out = f"{round(total, 2)}{(' ' + unit) if unit else ''}"
    return out

---

### Register Tools and Define Parsers

Set up a registry for dispatch. Use regex patterns to parse the agent’s output into actions or answers.

In [ ]:
from typing import Optional, List, Tuple

KNOWN_ACTIONS = {
    "lookup_distance": lookup_distance,
    "calculate_travel_time": calculate_travel_time,
    "calculate_sum": calculate_sum,
}

ACTION_RE = re.compile(
    r"Action:\\s*(?P<name>\\w+)\\s*\[(?P<params>.*?)]\\s*PAUSE",
    re.IGNORECASE | re.DOTALL
)
ANSWER_RE = re.compile(r"Answer:\\s*(?P<answer>.+)", re.IGNORECASE | re.DOTALL)

def parse_action(text: str) -> Optional[Tuple[str, List[str]]]:
    m = ACTION_RE.search(text)
    if not m:
        return None
    name = m.group("name")
    raw = m.group("params").strip()
    params = [p.strip().strip(""'").strip() for p in raw.split(",")] if raw else []
    return name, params

def parse_answer(text: str) -> Optional[str]:
    m = ANSWER_RE.search(text)
    return m.group("answer").strip() if m else None

def validate_action(name: str, params: List[str]) -> bool:
    if name not in KNOWN_ACTIONS:
        raise ValueError(f"Unknown action: {name}")
    return True

---

### Build the Control Loop

This loops through thinking, acting, observing until you get a final answer or hit a turn limit. This prevents runaway agents.

In [ ]:
def run_agent_loop(question: str, max_turns: int = 10, verbose: bool = True) -> str:
    agent = Agent(system_prompt=REACT_SYSTEM_PROMPT, model="gpt-4o", temperature=0)
    last = agent(question)
    if verbose:
        print("TURN 1 - ASSISTANT\n", last, "\n")

    turn = 1
    while turn < max_turns:
        answer = parse_answer(last)
        if answer:
            if verbose:
                print("FINAL ANSWER\n", answer)
            return answer

        parsed = parse_action(last)
        if not parsed:
            if verbose:
                print("No action or answer detected. Stopping.")
            return "Unable to complete: no action or answer detected."

        name, params = parsed
        try:
            validate_action(name, params)
            tool = KNOWN_ACTIONS[name]
            result = tool(*params)
        except Exception as e:
            result = f"ERROR: {str(e)}"

        obs_msg = f"Observation: {result}"
        turn += 1
        last = agent(obs_msg)
        if verbose:
            print(f"TURN {turn} - OBSERVATION\n", obs_msg)
            print(f"TURN {turn} - ASSISTANT\n", last, "\n")

    if verbose:
        print("Max turns reached without final answer.")
    return "Unable to complete within turn limit."

---

## Run and Validate

Test the agent with a multi\-step question that requires two tool calls: distance lookup, then travel time calculation.

In [ ]:
if __name__ == "__main__":
    print(run_agent_loop("How long to drive from Montreal to Boston at 60 mph?", max_turns=8))

**Expected output:**

In [ ]:
TURN 1 - ASSISTANT
Thought: I need to find the distance from Montreal to Boston first.
Action: lookup_distance[Montreal, Boston]
PAUSE

TURN 2 - OBSERVATION
...

---

## Connecting Back to ReAct (the Paper)

Here is what *ReAct: Synergizing Reasoning and Acting in Language Models* teaches you, and how it shaped this tutorial:

* The paper shows that when you interleave reasoning and action you get much more robust behavior. Reasoning lets the model plan. Actions let it get grounded information. Observations let it correct and update what it thought. ([arxiv.org](https://arxiv.org/abs/2210.03629?utm_source=openai))
* ReAct outperforms reasoning\-only (Chain\-of\-Thought) and acting\-only approaches on tasks like HotpotQA and FEVER. It reduces hallucinations and error propagation by using external sources. ([arxiv.org](https://arxiv.org/abs/2210.03629?utm_source=openai))
* Its format is similar here: *Thought → Action → Observation → Thought → … Answer*. That gives you both interpretability and control.
* You’ll see in the frameworks you already use that they adopt very similar contracts: they make you define tool signatures, control loops, stopping criteria, etc. This tutorial reproduces those pieces explicitly so you see them.

---

You’re now set up to experiment, tweak formats, add tools, or change the logic. Doing this by hand teaches you what the frameworks automate. That knowledge will make you a better builder, better debugger, and better decision\-maker about which abstractions to introduce.